In [1]:
import pandas as pd
import numpy as np
import os
import gensim
import sklearn
from sklearn import ensemble,svm,neural_network,discriminant_analysis
from sklearn.metrics import roc_curve,auc,precision_recall_curve
from matplotlib import pyplot as plt

with open("sent_corpus.csv", "r") as sent_file:
    lines = sent_file.read().split("\n")

rows = [line.split(",") for line in lines if line]
rows = [row[:3] + [",".join(row[3:])] for row in rows]
# remove document start character
rows[0][0] = rows[0][0][1:]
sentDf_cols = ['ItemID', 'Sentiment', 'SentimentSource', 'SentimentText']
sentDf = pd.DataFrame(rows[1:],columns=sentDf_cols)
# print(sentDf.columns.values)
sentDf[["ItemID","Sentiment"]] = sentDf[["ItemID","Sentiment"]].astype(int)
sentDf["SentimentText"] = sentDf["SentimentText"].apply(lambda text: text.split())
## -*- coding: utf-8 -*-
curdir = os.getcwd()
tweet_path = curdir + "/crawler_v1/code/tweettext.txt"
# print(tweet_path)

tweets = []
tweet_file = open(tweet_path, "rb")
for line in tweet_file:
#    print(line)
#    line = line_pre.decode('utf-8')
    tweet = ""
    if(line[:2] == "RT"):
        char_loc = 0
        for char in line:
            char_loc += 1
            if(char==":"):
                break

        tweet = line[-(len(line)-char_loc):]
    else:
        tweet = line
    tweets.append(tweet)

#[tweet.decode('utf-8') for tweet in tweets]
# print(tweets)
# print("done")


## 1-D searches on hyperparameters 

In [2]:
sentDf.head(10)

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,"[is, so, sad, for, my, APL, friend.............]"
1,2,0,Sentiment140,"[I, missed, the, New, Moon, trailer...]"
2,3,1,Sentiment140,"[omg, its, already, 7:30, :O]"
3,4,0,Sentiment140,"[.., Omgaga., Im, sooo, im, gunna, CRy., I've,..."
4,5,0,Sentiment140,"[i, think, mi, bf, is, cheating, on, me!!!, T_T]"
5,6,0,Sentiment140,"[or, i, just, worry, too, much?]"
6,7,1,Sentiment140,"[Juuuuuuuuuuuuuuuuussssst, Chillin!!]"
7,8,0,Sentiment140,"[Sunny, Again, Work, Tomorrow, :-|, TV, Tonight]"
8,9,1,Sentiment140,"[handed, in, my, uniform, today, ., i, miss, y..."
9,10,1,Sentiment140,"[hmmmm...., i, wonder, how, she, my, number, @-)]"


In [3]:
size_list = [100, 110, 120, 130, 140, 150]

In [4]:
# convertion i = (x - 100)/10 

In [5]:
Res = np.zeros((10,6))

In [6]:
from sklearn.model_selection import KFold

In [16]:
%%time
# test effect of 
count=0 
sentdat = sentDf["SentimentText"]
size_s = 100
window_s = 5 
total_f1 = 0 
kf = KFold(n_splits=5, shuffle=True)
for train_indeces, test_indeces in kf.split(sentdat):
    train_X = sentDf["SentimentText"].iloc[train_indeces]
    train_Y = sentDf["Sentiment"].iloc[train_indeces]
    test_X = sentDf["SentimentText"].iloc[test_indeces]
    test_Y = sentDf["Sentiment"].iloc[test_indeces]
    # Ns = int(1e5)
    # training set word2vec 
    w2vM1 = gensim.models.Word2Vec(train_X, size=size_s, window=window_s)
    tvecs_train = np.array([np.array([w2vM1[t] if t in w2vM1
                                else np.zeros((size_s,))                               
                            for t in twt]).mean(axis=0)
                     for twt in train_X])
    # test set word2vec 
    w2vM2 = gensim.models.Word2Vec(test_X, size=size_s, window=window_s)
    tvecs_test = np.array([np.array([w2vM2[t] if t in w2vM2
                                else np.zeros((size_s,))                               
                            for t in twt]).mean(axis=0)
                     for twt in test_X])
   
    # training
    mlp = sklearn.neural_network.MLPClassifier()
    mlp.fit(tvecs_train,train_Y)
    yhat = mlp.predict(tvecs_test)
    accu = np.mean(yhat == test_Y)
    prec = np.mean(test_Y[yhat == 1])
    recl = np.mean(yhat[test_Y == 1])
    f1 = 2 * prec * recl / (prec + recl)
    count+=1 
    print(count)
    print(f1)
    total_f1 += f1 
avgf1 = total_f1 / 5 
print(avgf1)

1
0.681203713994
2
0.579511057872
3
0.656619721451
4
0.543026947814
5
0.65166750766
0.622405789758
CPU times: user 1h 1min 32s, sys: 1min 4s, total: 1h 2min 36s
Wall time: 31min 58s


In [17]:
%%time 
#examine the effect of window size first 
res1 = [] 
sentdat = sentDf["SentimentText"]
for window_s in range(1, 11):
    #for size_s in size_list:
    size_s = 100 
    total_f1 = 0 
    kf = KFold(n_splits=5, shuffle=True)
    for train_indeces, test_indeces in kf.split(sentdat):
        train_X = sentDf["SentimentText"].iloc[train_indeces]
        train_Y = sentDf["Sentiment"].iloc[train_indeces]
        test_X = sentDf["SentimentText"].iloc[test_indeces]
        test_Y = sentDf["Sentiment"].iloc[test_indeces]
            # Ns = int(1e5)
            # training set word2vec 
        w2vM1 = gensim.models.Word2Vec(train_X, size=size_s, window=window_s)
        tvecs_train = np.array([np.array([w2vM1[t] if t in w2vM1
                                            else np.zeros((size_s,))                               
                                            for t in twt]).mean(axis=0)
                                            for twt in train_X])
            # test set word2vec 
        w2vM2 = gensim.models.Word2Vec(test_X, size=size_s, window=window_s)
        tvecs_test = np.array([np.array([w2vM2[t] if t in w2vM2
                                    else np.zeros((size_s,))                               
                                    for t in twt]).mean(axis=0)
                                    for twt in test_X])
   
            # training
        mlp = sklearn.neural_network.MLPClassifier()
        mlp.fit(tvecs_train,train_Y)
        yhat = mlp.predict(tvecs_test)
        accu = np.mean(yhat == test_Y)
        prec = np.mean(test_Y[yhat == 1])
        recl = np.mean(yhat[test_Y == 1])
        f1 = 2 * prec * recl / (prec + recl)
        print(f1)
        total_f1 += f1 
    avgf1 = total_f1 / 5 
    print(avgf1)
        # store the f1 score in the grid as the metrics
    res1.append(avgf1) 

0.663737209231
0.538535024603
0.670740934133
0.631841331969
0.638166431339
0.628604186255
0.493301658054
0.585393313771
0.485609348365
0.687707596008
0.664056704622
0.583213724164
0.582441874745
0.651941120673
0.608930424287
0.545765688344
0.658151959279
0.609446213466
0.674993293424
0.569633470663
0.548441986199
0.666898814463
0.659139695858
0.623821452122
0.579081649826
0.671334701275
0.510108444115
0.469849001048
0.658212037438
0.57771716674
0.66177614817
0.540865730673
0.633191755617
0.620580691095
0.517409068965
0.594764678904
0.677866061294
0.436807297804
0.46907341269
0.632108133744
0.534687150012
0.550108411109
0.427082826765
0.633430252726
0.631802803287
0.47831762263
0.487778873364
0.531682475754
0.599812490985
0.550565627954
0.544030352677
0.406612851735
0.51267209372
0.522738683415
0.505502410715
0.483879166498
0.553865723853
0.416524384088
0.327573645243
0.457469066079
CPU times: user 10h 19min 17s, sys: 12min 34s, total: 10h 31min 51s
Wall time: 5h 23min 56s


In [24]:
res1

[0.6286041862550471,
 0.58321372416399864,
 0.609446213465622,
 0.62382145212159301,
 0.57771716674034723,
 0.59476467890394968,
 0.55010841110874997,
 0.53168247575417127,
 0.52273868341450636,
 0.45746906607924875]

In [7]:
%%time 
#examine the effect of size along 
res2 = [] 
sentdat = sentDf["SentimentText"]
window_s = 1 
for size_s in size_list: 
    total_f1 = 0 
    kf = KFold(n_splits=5, shuffle=True)
    for train_indeces, test_indeces in kf.split(sentdat):
        train_X = sentDf["SentimentText"].iloc[train_indeces]
        train_Y = sentDf["Sentiment"].iloc[train_indeces]
        test_X = sentDf["SentimentText"].iloc[test_indeces]
        test_Y = sentDf["Sentiment"].iloc[test_indeces]
            # Ns = int(1e5)
            # training set word2vec 
        w2vM1 = gensim.models.Word2Vec(train_X, size=size_s, window=window_s)
        tvecs_train = np.array([np.array([w2vM1[t] if t in w2vM1
                                            else np.zeros((size_s,))                               
                                            for t in twt]).mean(axis=0)
                                            for twt in train_X])
            # test set word2vec 
        w2vM2 = gensim.models.Word2Vec(test_X, size=size_s, window=window_s)
        tvecs_test = np.array([np.array([w2vM2[t] if t in w2vM2
                                    else np.zeros((size_s,))                               
                                    for t in twt]).mean(axis=0)
                                    for twt in test_X])
   
            # training
        mlp = sklearn.neural_network.MLPClassifier()
        mlp.fit(tvecs_train,train_Y)
        yhat = mlp.predict(tvecs_test)
        accu = np.mean(yhat == test_Y)
        prec = np.mean(test_Y[yhat == 1])
        recl = np.mean(yhat[test_Y == 1])
        f1 = 2 * prec * recl / (prec + recl)
       # print(f1)
        total_f1 += f1 
    avgf1 = total_f1 / 5 
    print(avgf1)
        # store the f1 score in the grid as the metrics
    res2.append(avgf1) 

0.583316239143
0.601730913933
0.571610200031
0.642035005679
0.656939492252
0.536194690559
CPU times: user 7h 57min 31s, sys: 9min 50s, total: 8h 7min 21s
Wall time: 4h 11min 11s


In [8]:
res2

[0.58331623914256769,
 0.60173091393263833,
 0.57161020003090379,
 0.64203500567903926,
 0.65693949225247883,
 0.53619469055893942]

In [10]:
max_size_s = res2.index(max(res2))
print(size_list[max_size_s])

140


In [7]:
%%time 
# candidate range: wind_s = 1 or 4, size = 130 or 40 
# find the f1 score for cells intersecting the best candidate 
wind_cand = [1, 4]
size_cand = [130, 140]
#examine the effect of window size first 
res3 = [] 
sentdat = sentDf["SentimentText"]
for window_s in wind_cand:
    for size_s in size_cand:  
        total_f1 = 0 
        kf = KFold(n_splits=5, shuffle=True)
        for train_indeces, test_indeces in kf.split(sentdat):
            train_X = sentDf["SentimentText"].iloc[train_indeces]
            train_Y = sentDf["Sentiment"].iloc[train_indeces]
            test_X = sentDf["SentimentText"].iloc[test_indeces]
            test_Y = sentDf["Sentiment"].iloc[test_indeces]
            # Ns = int(1e5)
            # training set word2vec 
            w2vM1 = gensim.models.Word2Vec(train_X, size=size_s, window=window_s)
            tvecs_train = np.array([np.array([w2vM1[t] if t in w2vM1
                                            else np.zeros((size_s,))                               
                                            for t in twt]).mean(axis=0)
                                            for twt in train_X])
            # test set word2vec 
            w2vM2 = gensim.models.Word2Vec(test_X, size=size_s, window=window_s)
            tvecs_test = np.array([np.array([w2vM2[t] if t in w2vM2
                                    else np.zeros((size_s,))                               
                                    for t in twt]).mean(axis=0)
                                    for twt in test_X])
   
            # training
            mlp = sklearn.neural_network.MLPClassifier()
            mlp.fit(tvecs_train,train_Y)
            yhat = mlp.predict(tvecs_test)
            accu = np.mean(yhat == test_Y)
            prec = np.mean(test_Y[yhat == 1])
            recl = np.mean(yhat[test_Y == 1])
            f1 = 2 * prec * recl / (prec + recl)
            print(f1)
            total_f1 += f1 
        avgf1 = total_f1 / 5 
        print(avgf1)
        # store the f1 score in the grid as the metrics
        res3.append(avgf1) 

0.528952238453
0.648368169868
0.655557036587
0.361017480227
0.494371091608
0.537653203349
0.670917958205
0.675034605239
0.635727453439
0.570370968215
0.623449069719
0.635100010964
0.678533071898
0.576089504088
0.294803248476
0.659547696842
0.662212515733
0.574237207407
0.586663439103
0.626016308341
0.369858088931
0.67184562947
0.674605077785
0.585797708726
CPU times: user 5h 4min 33s, sys: 7min 2s, total: 5h 11min 36s
Wall time: 2h 40min 59s


In [8]:
res3

[0.53765320334871325,
 0.63510001096353474,
 0.57423720740726847,
 0.58579770872625048]

In [1]:
# 1 and 140 are the best values for window size and size 

In [9]:
%%time 
# testing more hyperparameters 
# add two more hyperparameters: min_count = [1,2,3,4,5,6,7,8,9]
# iter = [1,2,3,4,5,6,7,8,9]
#examine the effect of mincount alone  
res4 = [] 
sentdat = sentDf["SentimentText"]
for min_count in range(1,11): 
    total_f1 = 0 
    kf = KFold(n_splits=5, shuffle=True)
    for train_indeces, test_indeces in kf.split(sentdat):
        train_X = sentDf["SentimentText"].iloc[train_indeces]
        train_Y = sentDf["Sentiment"].iloc[train_indeces]
        test_X = sentDf["SentimentText"].iloc[test_indeces]
        test_Y = sentDf["Sentiment"].iloc[test_indeces]
            # Ns = int(1e5)
            # training set word2vec 
        w2vM1 = gensim.models.Word2Vec(train_X, min_count=min_count)
        tvecs_train = np.array([np.array([w2vM1[t] if t in w2vM1
                                            else np.zeros((100,))                               
                                            for t in twt]).mean(axis=0)
                                            for twt in train_X])
            # test set word2vec 
        w2vM2 = gensim.models.Word2Vec(test_X, min_count=min_count)
        tvecs_test = np.array([np.array([w2vM2[t] if t in w2vM2
                                    else np.zeros((100,))                               
                                    for t in twt]).mean(axis=0)
                                    for twt in test_X])
   
            # training
        mlp = sklearn.neural_network.MLPClassifier()
        mlp.fit(tvecs_train,train_Y)
        yhat = mlp.predict(tvecs_test)
        accu = np.mean(yhat == test_Y)
        prec = np.mean(test_Y[yhat == 1])
        recl = np.mean(yhat[test_Y == 1])
        f1 = 2 * prec * recl / (prec + recl)
       # print(f1)
        total_f1 += f1 
    avgf1 = total_f1 / 5 
    print(avgf1)
        # store the f1 score in the grid as the metrics
    res4.append(avgf1) 

0.476345236134
0.471677950478
0.53690397829
0.587296214155
0.58041945421
0.450108135444
0.602641403302
0.555231877584
0.574968157667
0.587839002012
CPU times: user 10h 31min 23s, sys: 12min 36s, total: 10h 43min 59s
Wall time: 5h 30min 16s


In [7]:
%%time 
# testing more hyperparameters 
# add two more hyperparameters: min_count = [1,2,3,4,5,6,7,8,9]
# iter = [1,2,3,4,5,6,7,8,9]
#examine the effect of size along 
res5 = [] 
sentdat = sentDf["SentimentText"]
for iteration in range(1,11): 
    total_f1 = 0 
    kf = KFold(n_splits=5, shuffle=True)
    for train_indeces, test_indeces in kf.split(sentdat):
        train_X = sentDf["SentimentText"].iloc[train_indeces]
        train_Y = sentDf["Sentiment"].iloc[train_indeces]
        test_X = sentDf["SentimentText"].iloc[test_indeces]
        test_Y = sentDf["Sentiment"].iloc[test_indeces]
            # Ns = int(1e5)
            # training set word2vec 
        w2vM1 = gensim.models.Word2Vec(train_X, iter=iteration)
        tvecs_train = np.array([np.array([w2vM1[t] if t in w2vM1
                                            else np.zeros((100,))                               
                                            for t in twt]).mean(axis=0)
                                            for twt in train_X])
            # test set word2vec 
        w2vM2 = gensim.models.Word2Vec(test_X, iter=iteration)
        tvecs_test = np.array([np.array([w2vM2[t] if t in w2vM2
                                    else np.zeros((100,))                               
                                    for t in twt]).mean(axis=0)
                                    for twt in test_X])
   
            # training
        mlp = sklearn.neural_network.MLPClassifier()
        mlp.fit(tvecs_train,train_Y)
        yhat = mlp.predict(tvecs_test)
        accu = np.mean(yhat == test_Y)
        prec = np.mean(test_Y[yhat == 1])
        recl = np.mean(yhat[test_Y == 1])
        f1 = 2 * prec * recl / (prec + recl)
       # print(f1)
        total_f1 += f1 
    avgf1 = total_f1 / 5 
    print(avgf1)
        # store the f1 score in the grid as the metrics
    res5.append(avgf1) 

0.226380788676
0.422943963953
0.441420007169
0.562868655333
0.585407691429
0.544513815189
0.604352710173
0.5884048094
0.61012185828
0.571130876475
CPU times: user 11h 6min 30s, sys: 13min 5s, total: 11h 19min 35s
Wall time: 5h 46min 24s


## Summary for 1-D search of hyperparameters
hyperparameters tested are: 
* Size: candidates are picked from the list [100,110,120,130,140,150]. and the best result is when size=100 
* Window size: candidates are picked from 1 to 10, and the best result is wind_size = 1 
* min_count: candidates are picked from 1 to 10, and the best result is when min_count = 7 
* iter: candidates are picked from 1 to 10, and the best result is for iter=9

The rest to test is around the neighborhood of the tentative optimal point (140, 1, 7, 9). The neighbor contains the points: 
(130 or 140 or 150, 1 or 2, 6 or 7 or 8, 8 or 9 or 10), a total of 54 points.

Since 10 cases take around 6 hours to test, 54 cases take more than 1 day to finish. Therefore, need another reduction in candidates: for 1-D scan in wind_size, 2 performs much worse, so omit 2 and take the second best performer 4. for 1-D scan on size, the result from 130 is much better than 150, so omit 150. For 1-D scan on min_count, 6 performs almost the worst in the candidates, so omit 6. For 1-D scan on iteration, since the variation is low, omit 8. The remaining candidates are: 
(130 or 140, 1 or 4, 7 or 8, 9 or 10), a total of 16 points. 

Moreoever, when setting value to 130, the warning of "setting size to a multiple of 4 for better performance" is generated, therefore, size candidates are set to the cloest multiple of 4 to 130 in the range (130,140), and results are (132, 140) 

In [13]:
# The neighbor list
cand_list = [[132, 1, 7, 9], [132, 1, 7, 10], [132, 1, 8, 9], [132, 1, 8, 10],
             [132, 4, 7, 9], [132, 4, 7, 10], [132, 4, 8, 9], [132, 4, 8, 10],
             [140, 1, 7, 9], [140, 1, 7, 10], [140, 1, 8, 9], [140, 1, 8, 10],
             [140, 4, 7, 9], [140, 4, 7, 10], [140, 4, 8, 9], [140, 4, 8, 10]]


In [14]:
%%time

final_res = [] 
sentdat = sentDf["SentimentText"]
for elem_tuple in cand_list: 
    total_f1 = 0 
    kf = KFold(n_splits=5, shuffle=True)
    for train_indeces, test_indeces in kf.split(sentdat):
        train_X = sentDf["SentimentText"].iloc[train_indeces]
        train_Y = sentDf["Sentiment"].iloc[train_indeces]
        test_X = sentDf["SentimentText"].iloc[test_indeces]
        test_Y = sentDf["Sentiment"].iloc[test_indeces]
        size = elem_tuple[0]
        wind_size = elem_tuple[1]
        min_count = elem_tuple[2]
        iteration = elem_tuple[3]
        #print("before the word2vec call")
        w2vM1 = gensim.models.Word2Vec(train_X, size=size, window=wind_size, min_count=min_count, iter=iteration)
        tvecs_train = np.array([np.array([w2vM1[t] if t in w2vM1
                                            else np.zeros((size,))                               
                                            for t in twt]).mean(axis=0)
                                            for twt in train_X])
            # test set word2vec 
        w2vM2 = gensim.models.Word2Vec(test_X, size=size, window=wind_size, min_count=min_count, iter=iteration)
        tvecs_test = np.array([np.array([w2vM2[t] if t in w2vM2
                                    else np.zeros((size,))                               
                                    for t in twt]).mean(axis=0)
                                    for twt in test_X])
   
            # training
        mlp = sklearn.neural_network.MLPClassifier()
        mlp.fit(tvecs_train,train_Y)
        yhat = mlp.predict(tvecs_test)
        accu = np.mean(yhat == test_Y)
        prec = np.mean(test_Y[yhat == 1])
        recl = np.mean(yhat[test_Y == 1])
        f1 = 2 * prec * recl / (prec + recl)
       # print(f1)
        total_f1 += f1 
    avgf1 = total_f1 / 5 
    print(avgf1)
        # store the f1 score in the grid as the metrics
    final_res.append(avgf1) 

0.63894002039
0.67790814957
0.648225117446
0.647054412866
0.58940117464
0.6443832454
0.612732949165
0.645904458341
0.661916432364
0.615625603522
0.664619296601
0.665382919372
0.64680585469
0.633735977261
0.579322116869
0.661123778066
CPU times: user 23h 41min 50s, sys: 30min 38s, total: 1d 12min 29s
Wall time: 11h 44min 24s
